In [1]:
# ========= 0. Imports & 定数 =========
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, f1_score, accuracy_score
import joblib

from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    f1_score, accuracy_score
)
import os, random
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split, GroupShuffleSplit
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, f1_score, accuracy_score
import numpy as np
import pandas as pd
import os, joblib
import re
from pandas.api.types import CategoricalDtype
import platform

# ==== Permutation Importance 用ユーティリティ ====
from sklearn.inspection import permutation_importance
from sklearn.metrics import get_scorer
import json, joblib
import hashlib

In [2]:
# === 入出力ディレクトリ設定 ===
# 入力は project_root/inputs
INPUT_DIR = os.path.join(os.getcwd(), "output")
# 出力は project_root/outputs
OUTPUT_DIR = os.path.join(os.getcwd(), "output_model_v1")
MODEL_DIR  = os.path.join(os.getcwd(), "model")
# ディレクトリが無ければ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("INPUT_DIR:", INPUT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("MODEL_DIR:", MODEL_DIR)


INPUT_DIR: /Users/okada1015/Desktop/マリサ/marisa/output
OUTPUT_DIR: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1
MODEL_DIR: /Users/okada1015/Desktop/マリサ/marisa/model


In [3]:

# ========= 1. 乱数固定（決定性の土台） =========
SEED = 42  # ←固定。変えれば分割・学習が変わるので注意
os.environ["PYTHONHASHSEED"] = "0"  # ※Jupyterではカーネル再起動が必要になる点に注意
random.seed(SEED)
np.random.seed(SEED)

# ========= 2. バージョン確認（任意） =========
print("Python:", platform.python_version())

Python: 3.13.5


In [4]:

# # ========= 3. 保存先 =========
# MODEL_DIR  = "/Users/okada1015/Desktop/マリサ/model"
# OUTPUT_DIR = "/Users/okada1015/Desktop/マリサ/output"
# os.makedirs(MODEL_DIR, exist_ok=True)
# os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:


# ========= 4. データ読み込み =========
# 北→南リスト（順序固定のため）
PREF_NORTH_TO_SOUTH = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県",
    "茨城県", "栃木県", "群馬県", "埼玉県", "千葉県", "東京都", "神奈川県",
    "新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県",
    "岐阜県", "静岡県", "愛知県", "三重県", "滋賀県", "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県",
    "鳥取県", "島根県", "岡山県", "広島県", "山口県",
    "徳島県", "香川県", "愛媛県", "高知県",
    "福岡県", "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", "鹿児島県",
    "沖縄県"
]
dtype_pref = CategoricalDtype(categories=PREF_NORTH_TO_SOUTH, ordered=True)

#マンション
df_m = pd.read_csv(os.path.join(INPUT_DIR, "3.ASSESSMENT_REPORTS_WITH_FLG_EDA_mansion.csv"),
                   encoding="utf-8-sig", dtype={"PREFECTURE": "string"})
#土地
df_l = pd.read_csv(os.path.join(INPUT_DIR, "3.ASSESSMENT_REPORTS_WITH_FLG_EDA_land.csv"),
                   encoding="utf-8-sig", dtype={"PREFECTURE": "string"})
#戸建
df_h = pd.read_csv(os.path.join(INPUT_DIR, "3.ASSESSMENT_REPORTS_WITH_FLG_EDA_house.csv"),
                   encoding="utf-8-sig", dtype={"PREFECTURE": "string"})

df_m["PREFECTURE"] = df_m["PREFECTURE"].astype(dtype_pref)
df_l["PREFECTURE"] = df_l["PREFECTURE"].astype(dtype_pref)
df_h["PREFECTURE"] = df_h["PREFECTURE"].astype(dtype_pref)
print("shapes:", df_m.shape, df_l.shape, df_h.shape)

/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/3809316497.py:16: DtypeWarning: Columns (21,33,44,45,75,77,78,91,92,93,94,100,106,110,111,120,122,155,159,209,210,211,221,224,225,226,227,228,230,238,240,241,242,243,244,245,247,255,257,258,259,260,261,262,263,264,307,308,311,312,314,315,317,335,336,337,341,343,346,354,355,356,357,359,383) have mixed types. Specify dtype option on import or set low_memory=False.
  df_m = pd.read_csv(os.path.join(INPUT_DIR, "3.ASSESSMENT_REPORTS_WITH_FLG_EDA_mansion.csv"),
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/3809316497.py:19: DtypeWarning: Columns (28,33,44,45,46,54,55,56,57,58,59,68,70,74,77,78,91,92,93,94,100,104,105,106,107,108,120,121,155,159,209,210,211,221,224,226,227,228,230,238,240,241,242,243,244,245,246,247,254,255,257,258,259,260,261,262,263,302,303,307,308,309,310,311,312,314,315,317,335,336,337,341,343,346,354,355,356,357,359) have mixed types. Specify dtype option on import or set low_memory=False

shapes: (113031, 394) (24658, 393) (69099, 395)


In [6]:

# ========= 5. 特徴量定義（あなたの元コードそのまま） =========
# --- マンション (property_kind = 1) ---
feature_cols_m = [
'FLOOR_NUMBER_NUM','OWNER_SPACE_NUM','DIRECTION','DIRECTION_SIN','DIRECTION_COS','PREFECTURE','REGION',
'LAND_PRIVILEGE_FLG','USAGE_AREA_REP','STRUCTURE_CAT','BUILDING_AGE','BUILDING_AGE_BIN','ABOUT_FLOOR_NUM',
'ABOUT_UNIT_AMOUNT_NUM','ASSESS_SELL_PRICE_NUM','ASSESS_SELL_SQUARE_PRICE_NUM','ASSESS_SELL_TSUBO_PRICE_NUM',
'ASSESS_SELL_TERM_NUM','VISIT_ASSESSMENT_FLG','ASSESS_PURCHASE_TERM_NUM','RENT_PRICE_FLG','INDICATE_COVER_STATISTICS',
'INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY','INDICATE_PRICE_HISTORY_GRAPH','INDICATE_PRICE_HISTORY_SELF_COMPARE',
'INDICATE_PRICE_HISTORY_AREA_COMPARE','INDICATE_AREA_HUMAN','INDICATE_AREA_HUMAN_AGE_GRAPH',
'INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET','INDICATE_MARKET_SIZE','INDICATE_MARKET_SELL_TERM',
'INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT','INTEREST_COMMENT_FLG','INDICATE_MEDIATION_CONTRACT',
'EXPENSE1_SELL_PRICE_NUM','EXPENSE1_COMMISSION_RATE','EXPENSE1_COMMISSION_RATE_FLG','SP_READ_FLG','TUTORIAL_SP',
'READ_FLG','READ_COUNT','CONTENT_COUNT','CONTENT_COUNT_FLG','TXT_NOTIFY_READ_AT','PRICE_DETAIL_COUNT',
'MEDIATION_BUY_COUNT','MARKET_CONDITION_COUNT','CONSULTATION_MOVE_COUNT','TAX_ADVICE_COUNT','SELLING_EXPENSE_COUNT',
'THANKS_COUNT','THX_NOTIFY_READ_AT_COUNT','HOPE_TEL_FLG_COUNT','TEL_NOTIFY_READ_AT_COUNT','MEETING_FLG_COUNT','MTG_NOTIFY_READ_AT_COUNT'
]
category_m = [
'DIRECTION','PREFECTURE','REGION','LAND_PRIVILEGE_FLG','USAGE_AREA_REP','STRUCTURE_CAT','BUILDING_AGE_BIN',
'VISIT_ASSESSMENT_FLG','RENT_PRICE_FLG','INDICATE_COVER_STATISTICS','INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY',
'INDICATE_PRICE_HISTORY_GRAPH','INDICATE_PRICE_HISTORY_SELF_COMPARE','INDICATE_PRICE_HISTORY_AREA_COMPARE',
'INDICATE_AREA_HUMAN','INDICATE_AREA_HUMAN_AGE_GRAPH','INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET',
'INDICATE_MARKET_SIZE','INDICATE_MARKET_SELL_TERM','INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT',
'INTEREST_COMMENT_FLG','INDICATE_MEDIATION_CONTRACT','EXPENSE1_COMMISSION_RATE_FLG','SP_READ_FLG','TUTORIAL_SP',
'READ_FLG','CONTENT_COUNT_FLG','TXT_NOTIFY_READ_AT','THX_NOTIFY_READ_AT_COUNT','HOPE_TEL_FLG_COUNT',
'TEL_NOTIFY_READ_AT_COUNT','MEETING_FLG_COUNT','MTG_NOTIFY_READ_AT_COUNT',
]

# --- 土地 (property_kind = 2) ---
feature_cols_l = [
'LAND_SPACE_NUM','PREFECTURE','REGION','LAND_PRIVILEGE_FLG','USAGE_AREA_REP','ASSESS_SELL_PRICE_NUM',
'ASSESS_SELL_SQUARE_PRICE_NUM','ASSESS_SELL_TSUBO_PRICE_NUM','ASSESS_SELL_TERM_NUM','VISIT_ASSESSMENT_FLG',
'ASSESS_PURCHASE_TERM_NUM','RENT_PRICE_FLG','INDICATE_COVER_STATISTICS','INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY',
'INDICATE_PRICE_HISTORY_GRAPH','INDICATE_PRICE_HISTORY_SELF_COMPARE','INDICATE_PRICE_HISTORY_AREA_COMPARE',
'INDICATE_AREA_HUMAN','INDICATE_AREA_HUMAN_AGE_GRAPH','INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET',
'INDICATE_MARKET_SIZE','INDICATE_MARKET_SELL_TERM','INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT',
'INTEREST_COMMENT_FLG','INDICATE_MEDIATION_CONTRACT','EXPENSE1_SELL_PRICE_NUM','EXPENSE1_COMMISSION_RATE',
'EXPENSE1_COMMISSION_RATE_FLG','SP_READ_FLG','TUTORIAL_SP','READ_FLG','READ_COUNT','CONTENT_COUNT','CONTENT_COUNT_FLG',
'TXT_NOTIFY_READ_AT','PRICE_DETAIL_COUNT','MEDIATION_BUY_COUNT','MARKET_CONDITION_COUNT','CONSULTATION_MOVE_COUNT',
'TAX_ADVICE_COUNT','SELLING_EXPENSE_COUNT','THANKS_COUNT','THX_NOTIFY_READ_AT_COUNT','HOPE_TEL_FLG_COUNT',
'TEL_NOTIFY_READ_AT_COUNT','MEETING_FLG_COUNT','MTG_NOTIFY_READ_AT_COUNT'
]
category_l = [
'PREFECTURE','REGION','LAND_PRIVILEGE_FLG','USAGE_AREA_REP','VISIT_ASSESSMENT_FLG','RENT_PRICE_FLG',
'INDICATE_COVER_STATISTICS','INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY','INDICATE_PRICE_HISTORY_GRAPH',
'INDICATE_PRICE_HISTORY_SELF_COMPARE','INDICATE_PRICE_HISTORY_AREA_COMPARE','INDICATE_AREA_HUMAN',
'INDICATE_AREA_HUMAN_AGE_GRAPH','INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET',
'INDICATE_MARKET_SIZE','INDICATE_MARKET_SELL_TERM','INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT',
'INTEREST_COMMENT_FLG','INDICATE_MEDIATION_CONTRACT','EXPENSE1_COMMISSION_RATE_FLG','SP_READ_FLG','READ_FLG',
'CONTENT_COUNT_FLG','TXT_NOTIFY_READ_AT','HOPE_TEL_FLG_COUNT','TEL_NOTIFY_READ_AT_COUNT','MEETING_FLG_COUNT',
'MTG_NOTIFY_READ_AT_COUNT',
]

# --- 戸建 (property_kind = 3) ---
feature_cols_h = [
'LAND_SPACE_NUM','BUILDING_SPACE_NUM','PREFECTURE','REGION','LAND_PRIVILEGE_FLG','USAGE_AREA_REP','STRUCTURE_CAT',
'BUILDING_AGE','BUILDING_AGE_BIN','ABOUT_FLOOR_NUM','ASSESS_SELL_PRICE_NUM','ASSESS_SELL_SQUARE_PRICE_NUM',
'ASSESS_SELL_TSUBO_PRICE_NUM','ASSESS_SELL_TERM_NUM','VISIT_ASSESSMENT_FLG','ASSESS_PURCHASE_TERM_NUM','RENT_PRICE_FLG',
'INDICATE_COVER_STATISTICS','INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY','INDICATE_PRICE_HISTORY_GRAPH',
'INDICATE_PRICE_HISTORY_SELF_COMPARE','INDICATE_PRICE_HISTORY_AREA_COMPARE','INDICATE_AREA_HUMAN',
'INDICATE_AREA_HUMAN_AGE_GRAPH','INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET','INDICATE_MARKET_SIZE',
'INDICATE_MARKET_SELL_TERM','INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT','INTEREST_COMMENT_FLG',
'INDICATE_MEDIATION_CONTRACT','EXPENSE1_SELL_PRICE_NUM','EXPENSE1_COMMISSION_RATE','EXPENSE1_COMMISSION_RATE_FLG',
'SP_READ_FLG','TUTORIAL_SP','READ_FLG','READ_COUNT','CONTENT_COUNT','CONTENT_COUNT_FLG','TXT_NOTIFY_READ_AT',
'PRICE_DETAIL_COUNT','MEDIATION_BUY_COUNT','MARKET_CONDITION_COUNT','CONSULTATION_MOVE_COUNT','TAX_ADVICE_COUNT',
'SELLING_EXPENSE_COUNT','THANKS_COUNT','THX_NOTIFY_READ_AT_COUNT','HOPE_TEL_FLG_COUNT','TEL_NOTIFY_READ_AT_COUNT',
'MEETING_FLG_COUNT','MTG_NOTIFY_READ_AT_COUNT',
]
category_h = [
'PREFECTURE','REGION','LAND_PRIVILEGE_FLG','USAGE_AREA_REP','STRUCTURE_CAT','BUILDING_AGE_BIN','VISIT_ASSESSMENT_FLG',
'RENT_PRICE_FLG','INDICATE_COVER_STATISTICS','INDICATE_MARKET_REPORTS','INDICATE_PRICE_HISTORY',
'INDICATE_PRICE_HISTORY_GRAPH','INDICATE_PRICE_HISTORY_SELF_COMPARE','INDICATE_PRICE_HISTORY_AREA_COMPARE',
'INDICATE_AREA_HUMAN','INDICATE_AREA_HUMAN_AGE_GRAPH','INDICATE_AREA_HUMAN_TRANSITION_GRAPH','INDICATE_COVER_MARKET',
'INDICATE_MARKET_SIZE','INDICATE_MARKET_SELL_TERM','INDICATE_MORTGAGE_INTEREST','INDICATE_FLOATING_INTEREST_COMMENT',
'INTEREST_COMMENT_FLG','INDICATE_MEDIATION_CONTRACT','EXPENSE1_COMMISSION_RATE_FLG','SP_READ_FLG','TUTORIAL_SP',
'READ_FLG','CONTENT_COUNT_FLG','TXT_NOTIFY_READ_AT','HOPE_TEL_FLG_COUNT','TEL_NOTIFY_READ_AT_COUNT',
'MEETING_FLG_COUNT','MTG_NOTIFY_READ_AT_COUNT',
]

In [7]:
# ========= 6. マッピング =========
kind_to_df    = {1: df_m,             2: df_l,             3: df_h}
kind_to_feats = {1: feature_cols_m,   2: feature_cols_l,   3: feature_cols_h}
kind_to_cats  = {1: category_m,       2: category_l,       3: category_h}

TARGET    = "成約フラグ"
GROUP_COL = None  # 例: "C_ID" を使うなら "C_ID"

In [8]:
# ========= 7. 決定性ヘルパー =========
def normalize_categories(df: pd.DataFrame, categorical_cols):
    """
    指定列を必ず category 化 → 未使用カテゴリ削除 → 水準の安定化（昇順 or 文字列昇順）
    """
    g = df.copy()
    for c in categorical_cols:
        if c not in g.columns:
            continue
        # 何型でもまず category 化（0/1や*_COUNTなど数値でも）
        g[c] = g[c].astype("category")
        # 未使用カテゴリを落とす
        g[c] = g[c].cat.remove_unused_categories()
        # カテゴリ水準を安定化（mixed型は文字列化してソート）
        cats = list(g[c].cat.categories)
        try:
            cats_sorted = sorted(cats)
        except TypeError:
            cats_sorted = sorted(map(str, cats))
        g[c] = g[c].cat.set_categories(cats_sorted, ordered=False)
    return g


def fingerprint_features(dfX: pd.DataFrame, cat_cols):
    meta = {
        "columns": list(dfX.columns),
        "dtypes": {c: str(dfX[c].dtype) for c in dfX.columns},
        "cat_cols": list(cat_cols),
        "cat_levels": {
            c: (list(dfX[c].cat.categories) if (c in dfX.columns and pd.api.types.is_categorical_dtype(dfX[c])) else None)
            for c in dfX.columns
        },
    }
    s = json.dumps(meta, ensure_ascii=False, sort_keys=True, default=str)
    return hashlib.md5(s.encode()).hexdigest(), meta

def model_hash(clf: LGBMClassifier):
    dump = clf.booster_.dump_model(num_iteration=clf.best_iteration_)
    s = json.dumps(dump, sort_keys=True)
    return hashlib.md5(s.encode()).hexdigest()

In [9]:
# ========= 8. 分割ユーティリティ（元コード準拠＋決定性） =========
def _build_cv(groups_available, n_splits=5, seed=SEED):
    return GroupKFold(n_splits) if groups_available else StratifiedKFold(n_splits, shuffle=True, random_state=seed)

def _split_train_test(df_kind, feature_cols, target_col, group_col=None, test_size=0.2, seed=SEED):
    X_full = df_kind.reindex(columns=feature_cols).copy()
    y_full = pd.Series(df_kind[target_col].astype(int).values, index=X_full.index, name=target_col)

    groups = (df_kind[group_col].reindex(X_full.index).values
              if group_col and group_col in df_kind.columns else None)

    if groups is not None:
        gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        tr_idx, te_idx = next(gss.split(X_full, y_full, groups))
    else:
        tr_idx, te_idx = train_test_split(
            np.arange(len(X_full)), test_size=test_size, random_state=seed, stratify=y_full.values
        )

    X_tr, X_te = X_full.iloc[tr_idx].copy(), X_full.iloc[te_idx].copy()
    y_tr, y_te = y_full.iloc[tr_idx].copy(), y_full.iloc[te_idx].copy()
    grp_tr = (df_kind[group_col].iloc[tr_idx].values if groups is not None else None)
    return X_tr, y_tr, grp_tr, X_te, y_te

In [10]:


# ========= 9. LightGBM 学習本体（決定性版） =========
def train_eval_lgbm_no_leak(df_kind, feature_cols, category_cols,
                            target_col="成約フラグ", group_col=None,
                            seed=SEED, n_splits=5, test_size=0.2):
   # ---- split: 先に学習/テストを分ける ----
    X_tr, y_tr, grp_tr, X_te, y_te = _split_train_test(
        df_kind, feature_cols, target_col, group_col, test_size, seed
    )

    # ---- カテゴリ前処理（ここで必ず category 化＆水準固定）----
    cat_cols = [c for c in category_cols if c in X_tr.columns]
    X_tr = normalize_categories(X_tr, cat_cols)
    X_te = normalize_categories(X_te, cat_cols)

    # ---- CV 設定 ----
    cv = _build_cv(grp_tr is not None, n_splits=n_splits, seed=seed)
    oof = pd.Series(np.zeros(len(X_tr), dtype=float), index=X_tr.index)

    # ---- 決定性最大化パラメータ ----
    params = dict(
        objective="binary",
        learning_rate=0.03,
        n_estimators=5000,                    # early_stopping で短縮
        num_leaves=31,
        # ランダム要素の排除（※再現性を最優先）
        subsample=1.0,                        # = bagging_fraction
        colsample_bytree=1.0,                # = feature_fraction
        # 全シード固定
        random_state=seed, bagging_seed=seed, feature_fraction_seed=seed,
        data_random_seed=seed, drop_seed=seed,
        # 決定論モード & 並列揺らぎ抑止
        deterministic=True,
        n_jobs=1,                             # = num_threads=1
        # クラス不均衡への配慮（元コード踏襲）
        class_weight="balanced",
        # 追加の安定化（列ワイズ学習を強制）
        force_col_wise=True,
        verbose=-1,
    )

    best_iters, imps = [], []
    splitter = (cv.split(X_tr, y_tr.values, grp_tr) if isinstance(cv, GroupKFold)
                else cv.split(X_tr, y_tr.values))

    for fold, (tr, va) in enumerate(splitter, 1):
        Xtr, Xva = X_tr.iloc[tr].copy(), X_tr.iloc[va].copy()
        ytr, yva = y_tr.iloc[tr].values, y_tr.iloc[va].values

        # 学習foldのカテゴリ水準を検証foldに合わせる（安定化）
        # 学習foldのカテゴリ水準を検証foldに合わせる（安定化）
        for c in cat_cols:
            if not pd.api.types.is_categorical_dtype(Xtr[c]):
                Xtr[c] = Xtr[c].astype("category")
            if not pd.api.types.is_categorical_dtype(Xva[c]):
                Xva[c] = Xva[c].astype("category")
            Xtr[c] = Xtr[c].cat.remove_unused_categories()
            Xva[c] = Xva[c].cat.set_categories(Xtr[c].cat.categories)


        model = LGBMClassifier(**params)
        model.fit(
            Xtr, ytr,
            eval_set=[(Xva, yva)],
            eval_metric="auc",
            callbacks=[early_stopping(stopping_rounds=200), log_evaluation(0)]
        )

        oof.iloc[va] = model.predict_proba(Xva)[:, 1]
        best_iters.append(model.best_iteration_)
        imps.append(pd.DataFrame({
            "feature": X_tr.columns,
            "importance": model.booster_.feature_importance(importance_type="gain"),
            "fold": fold
        }))

    # ---- CV メトリクス ----
    y_true_tr = y_tr.values
    oof_pred  = oof.values
    cv_auc    = roc_auc_score(y_true_tr, oof_pred)
    cv_ap     = average_precision_score(y_true_tr, oof_pred)
    prec, rec, thr = precision_recall_curve(y_true_tr, oof_pred)
    f1s = 2 * prec * rec / (prec + rec + 1e-12)
    bi = int(np.nanargmax(f1s))
    best_thr = 0.5 if bi >= len(thr) else float(thr[bi])
    cv_f1    = f1_score(y_true_tr, (oof_pred >= best_thr).astype(int))
    cv_acc   = accuracy_score(y_true_tr, (oof_pred >= best_thr).astype(int))

    # ---- 全学習データで最終学習（CVのbest_iter平均を使用）----
    final_ne = int(np.clip(np.mean(best_iters), 100, params["n_estimators"]))
    final = LGBMClassifier(**{**params, "n_estimators": final_ne})

    # テスト側カテゴリを学習側カテゴリに合わせる
    for c in cat_cols:
        X_tr[c] = X_tr[c].cat.remove_unused_categories()
        X_te[c] = X_te[c].astype("category").cat.set_categories(X_tr[c].cat.categories)

    final.fit(X_tr, y_tr.values, callbacks=[log_evaluation(0)])

    # ---- テスト評価 ----
    proba_te = final.predict_proba(X_te)[:, 1]
    yhat_te  = (proba_te >= best_thr).astype(int)

    te_auc   = roc_auc_score(y_te.values, proba_te)
    te_ap    = average_precision_score(y_te.values, proba_te)
    te_f1    = f1_score(y_te.values, yhat_te)
    te_acc   = accuracy_score(y_te.values, yhat_te)

    # 重要度（CV平均）
    imp_mean = (pd.concat(imps).groupby("feature")["importance"]
                .mean().sort_values(ascending=False).reset_index())

    metrics_cv   = {"AUC": cv_auc, "PR_AUC": cv_ap, "F1": cv_f1, "ACC": cv_acc,
                    "best_thr": best_thr, "best_iters_mean": float(np.mean(best_iters))}
    metrics_test = {"AUC": te_auc, "PR_AUC": te_ap, "F1": te_f1, "ACC": te_acc,
                    "thr_used": best_thr, "n_estimators": final_ne}

    # 付加: 決定性検証用ハッシュ
    feat_hash, feat_meta = fingerprint_features(X_tr, cat_cols)
    mod_hash = model_hash(final)

    return final, metrics_cv, metrics_test, imp_mean, cat_cols, oof, (X_tr.index, X_te.index), (feat_hash, feat_meta, mod_hash)




In [11]:
# ========= 10. 学習→保存→評価 =========
summary = []
model_registry = {}

for kind in [1, 2, 3]:
    print(f"\n===== Training kind={kind} =====")
    df_kind      = kind_to_df[kind].copy()
    feature_cols = kind_to_feats[kind]
    category_cols= kind_to_cats[kind]

    model, metrics_cv, metrics_test, imp_df, cat_cols_final, oof_series, (idx_tr, idx_te), (feat_hash, feat_meta, mod_hash) = train_eval_lgbm_no_leak(
        df_kind, feature_cols, category_cols,
        target_col=TARGET, group_col=GROUP_COL, seed=SEED, n_splits=5, test_size=0.2
    )

    # === 分割情報とメタ保存（再現性トレース） ===
    pd.DataFrame({"train_index": list(idx_tr)}).to_csv(
        os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv"), index=False
    )
    pd.DataFrame({"test_index": list(idx_te)}).to_csv(
        os.path.join(OUTPUT_DIR, f"split_test_index_kind{kind}.csv"), index=False
    )
    meta = {
        "kind": kind,
        "cv_metrics": metrics_cv,
        "test_metrics": metrics_test,
        "features": feature_cols,
        "cat_cols": cat_cols_final,
        "best_threshold": metrics_cv["best_thr"],
        "n_estimators_final": metrics_test["n_estimators"],
        "feature_hash": feat_hash,
        "model_hash": mod_hash,
        "versions": {
            "python": platform.python_version(),
            "numpy": np.__version__,
            "pandas": pd.__version__,
        }
    }
    with open(os.path.join(OUTPUT_DIR, f"meta_kind{kind}.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    # === 予測保存（OOF と TEST） ===
    # OOF（学習データ内の検証時予測）
    pd.DataFrame({
        "index": oof_series.index,
        TARGET: df_kind.loc[oof_series.index, TARGET].astype(int).values,
        "oof_proba": oof_series.values,
        "oof_pred":  (oof_series.values >= metrics_cv["best_thr"]).astype(int)
    }).to_csv(os.path.join(OUTPUT_DIR, f"oof_kind{kind}.csv"), index=False, encoding="utf-8-sig")

    # TEST（完全ホールドアウト）
    X_te = df_kind.reindex(columns=feature_cols).loc[idx_te].copy()
    for c in cat_cols_final:
        if c in X_te.columns:
            X_te[c] = X_te[c].astype("category")
    for c in cat_cols_final:
        X_te[c] = X_te[c].cat.set_categories(df_kind.loc[idx_tr, c].astype("category").cat.categories)
    proba_te = model.predict_proba(X_te)[:, 1]

    kind_names = {1: "mansion", 2: "land", 3: "house"}
    kind_name = kind_names[kind]

    # ① テストデータ全件テーブル
    test_thr = metrics_cv["best_thr"]
    pred_te  = (proba_te >= test_thr).astype(int)
    y_te     = df_kind.loc[idx_te, TARGET].astype(int).values
    pd.DataFrame({
        "ID": df_kind.loc[idx_te, "ID"].values,
        TARGET: y_te,
        "proba": proba_te,
        "pred":  pred_te,
        "thr_used(閾値)": test_thr,
    }).to_csv(os.path.join(OUTPUT_DIR, f"test_scores_{kind_name}.csv"),
              index=False, encoding="utf-8-sig")

    # ② スコア帯分布 (0.1刻み)
    bins = np.linspace(0.0, 1.0, 11)
    labels = [f"{bins[i]:.1f}-{bins[i+1]:.1f}" for i in range(len(bins)-1)]
    counts, _ = np.histogram(proba_te, bins=bins)
    ratios = counts / max(len(proba_te), 1)
    hist_df = pd.DataFrame({
        "bin_left":  bins[:-1],
        "bin_right": bins[1:],
        "label":     labels,
        "count":     counts,
        "ratio":     ratios,
    })
    hist_df.to_csv(os.path.join(OUTPUT_DIR, f"test_score_hist_{kind_name}.csv"),
                   index=False, encoding="utf-8-sig")
    plt.figure(figsize=(8, 4))
    plt.bar(labels, counts)
    plt.title(f"Score histogram (test) - {kind_name}")
    plt.xlabel("Score band"); plt.ylabel("Count"); plt.xticks(rotation=0)
    for i, c in enumerate(counts):
        plt.text(i, c, str(c), ha="center", va="bottom", fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"test_score_hist_{kind_name}.png"), dpi=160)
    plt.close()

    # 重要度・特徴一覧
    imp_df.to_csv(os.path.join(OUTPUT_DIR, f"importance_kind{kind}.csv"),
                  index=False, encoding="utf-8-sig")
    pd.DataFrame({"feature": feature_cols}).to_csv(
        os.path.join(OUTPUT_DIR, f"features_kind{kind}.csv"),
        index=False, encoding="utf-8-sig"
    )

    # モデル保存
    joblib.dump(model, os.path.join(MODEL_DIR, f"lgbm_kind{kind}_v1.pkl"))

    # サマリー
    summary.append({
        "property_kind": kind,
        "cv_AUC": metrics_cv["AUC"], "cv_PR_AUC": metrics_cv["PR_AUC"],
        "cv_F1": metrics_cv["F1"], "cv_ACC": metrics_cv["ACC"],
        "thr_used(閾値)": metrics_cv["best_thr"],
        "test_AUC": metrics_test["AUC"], "test_PR_AUC": metrics_test["PR_AUC"],
        "test_F1": metrics_test["F1"], "test_ACC": metrics_test["ACC"],
        "feature_hash": feat_hash, "model_hash": mod_hash
    })

    model_registry[kind] = {"model": model, "features": feature_cols,
                            "cat_cols": cat_cols_final, "metrics_cv": metrics_cv,
                            "metrics_test": metrics_test}

# === 集計の表示 & 保存 ===
summary_df = pd.DataFrame(summary).sort_values("property_kind")
print(summary_df)
summary_df.to_csv(os.path.join(OUTPUT_DIR, "metrics_summary_by_kind.csv"),
                  index=False, encoding="utf-8-sig")

# === 一貫性チェック（OOF index / train-test 非交差） ===
for kind in model_registry.keys():
    df_kind = kind_to_df[kind]
    oof_path = os.path.join(OUTPUT_DIR, f"oof_kind{kind}.csv")
    oof_idx = pd.read_csv(oof_path)["index"].tolist()
    assert set(oof_idx).issubset(set(df_kind.index)), f"kind={kind}: OOFのindexが元dfに含まれていません"

    tr_idx = set(pd.read_csv(os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv"))["train_index"].tolist())
    te_idx = set(pd.read_csv(os.path.join(OUTPUT_DIR, f"split_test_index_kind{kind}.csv"))["test_index"].tolist())
    assert tr_idx.isdisjoint(te_idx), f"kind={kind}: train/test が交差しています"

print("\n✅ 完了：同一特徴量・同一SEEDなら毎回まったく同じモデル（model_hash）が得られます。")



===== Training kind=1 =====


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:52: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xtr[c]):
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:54: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xva[c]):


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[3149]	valid_0's auc: 0.698007	valid_0's binary_logloss: 0.498626


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:52: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xtr[c]):
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:54: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xva[c]):


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2741]	valid_0's auc: 0.695683	valid_0's binary_logloss: 0.506629


...

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1332]	valid_0's auc: 0.78773	valid_0's binary_logloss: 0.421024


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:52: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xtr[c]):
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/1010783313.py:54: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(Xva[c]):


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2035]	valid_0's auc: 0.784091	valid_0's binary_logloss: 0.391335


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/3305051690.py:30: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  c: (list(dfX[c].cat.categories) if (c in dfX.columns and pd.api.types.is_categorical_dtype(dfX[c])) else None)


   property_kind    cv_AUC  cv_PR_AUC     cv_F1    cv_ACC  thr_used(閾値)  \
0              1  0.699678   0.304062  0.340491  0.753063      0.501803   
1              2  0.679321   0.250676  0.307485  0.713424      0.499867   
2              3  0.788732   0.318970  0.355822  0.803433      0.547487   

   test_AUC  test_PR_AUC   test_F1  test_ACC  \
0  0.721797     0.330851  0.360217  0.739638   
1  0.689061     0.257519  0.313707  0.704582   
2  0.792587     0.338601  0.364999  0.791534   

                       feature_hash                        model_hash  
0  5e8dc78d1635ef1f6c9fe56ff261b3b7  ad86b0832baa3d87c4931bdac9102911  
1  fd480800ac602ac0093c6fa157b7657d  8a6dd726a79800c56ad6ad711a454e88  
2  14089385ccb5dad066aa3ae3315ad5a7  27b1cc9b5e9c7c3b702a43bd9dce2af1  

✅ 完了：同一特徴量・同一SEEDなら毎回まったく同じモデル（model_hash）が得られます。


# SHAP #

### 関数定義 ###

In [12]:
# FIXED: SHAP 入力前に object を category にそろえる安全バンド
def ensure_no_object_category(df):
    g = df.copy()
    for c in g.columns:
        if g[c].dtype == "object":
            g[c] = g[c].astype("category")
    return g


In [13]:
# FIXED: probability はやめて、raw + tree_path_dependent で安定運用
def make_shap_explainer_raw(model):
    import shap
    # feature_perturbation="tree_path_dependent" / background は渡さない
    return shap.TreeExplainer(
        model,
        feature_perturbation="tree_path_dependent",
        model_output="raw"
    )


In [14]:
import numpy as np, pandas as pd, shap, matplotlib.pyplot as plt, os, joblib

def make_X_for_kind(df_kind, feature_cols, category_cols):
    """学習と可視化の両方で使うXを返す。
       - 学習用: categoryにキャスト
       - 可視化用: すべて数値（カテゴリはcodes）"""
    X = df_kind.reindex(columns=feature_cols).copy()

    # まず object を category に（指定外にobjectが残っていても拾う）
    for c in X.columns:
        if (c in category_cols) or (X[c].dtype == "object"):
            X[c] = X[c].astype("category")

    return X

def to_numeric_for_plot(X):
    """SHAPの可視化向けに、カテゴリ列をcodesへ変換して全列数値にする"""
    Xp = X.copy()
    for c in Xp.columns:
        dt = Xp[c].dtype
        if str(dt).startswith("category"):
            Xp[c] = Xp[c].cat.codes
        elif dt == "object":
            # 念のため（残っていたらcategory→codes）
            Xp[c] = Xp[c].astype("category").cat.codes
    return Xp

def shap_all_plots(model, X, kind, sample_size=2000, seed=0,
                   topk_depend=5, do_interaction=False):
    """
    SHAP 可視化一括出力（RAWスケールで統一）
    - beeswarm / bar / TopK dependence
    """
    import numpy as np, pandas as pd, shap, matplotlib.pyplot as plt, os

    # FIXED: 念のため object→category に矯正
    X = ensure_no_object_category(X)  # FIXED

    # ---- サンプリング（再現性）----
    n = min(sample_size, len(X))
    idx = np.random.RandomState(seed).choice(X.index, size=n, replace=False)
    Xs = X.loc[idx]

    # 可視化用（カテゴリ→codes）
    def to_numeric_for_plot(df):
        g = df.copy()
        for c in g.columns:
            dt = g[c].dtype
            if str(dt).startswith("category"):
                g[c] = g[c].cat.codes
            elif dt == "object":
                g[c] = g[c].astype("category").cat.codes
        return g
    Xs_plot = to_numeric_for_plot(Xs)

    # ==== FIXED: raw + tree_path_dependent ====
    explainer = make_shap_explainer_raw(model)   # FIXED
    sv = explainer(Xs)                           # shap.Explanation (raw)

    # ---- 1) summary（beeswarm / bar）----
    plt.figure()
    shap.summary_plot(sv, Xs_plot, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"shap_beeswarm_kind{kind}.png"), dpi=180)
    plt.close()

    plt.figure()
    shap.summary_plot(sv, Xs_plot, plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"shap_bar_kind{kind}.png"), dpi=180)
    plt.close()

    # ---- 2) dependence（TopK）----
    mean_abs = np.abs(sv.values).mean(axis=0)
    top_feats = (pd.Series(mean_abs, index=Xs.columns)
                 .sort_values(ascending=False)
                 .head(topk_depend).index.tolist())

    for f in top_feats:
        plt.figure()
        shap.dependence_plot(
            ind=f,
            shap_values=sv.values,
            features=Xs_plot,
            feature_names=Xs_plot.columns,
            show=False
        )
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"shap_depend_{f}_kind{kind}.png"), dpi=180)
        plt.close()


def save_interaction_scores(model, X, kind, sample_size=2000, seed=SEED):
    """
    全特徴量ペアの相互作用スコアを安定な近似で算出して保存。
    score = |corr(SHAP[:, i], X_numeric[:, j])|
    """
    # ---- サンプリング（再現性） ----
    rng = np.random.RandomState(seed)
    idx = rng.choice(len(X), min(sample_size, len(X)), replace=False)
    Xs = X.iloc[idx].copy()

    # 可視化/相関計算用にカテゴリを codes に（数値化）
    def _to_numeric_for_plot(df):
        df2 = df.copy()
        for c in df2.columns:
            dt = df2[c].dtype
            if str(dt).startswith("category"):
                df2[c] = df2[c].cat.codes
            elif dt == "object":
                df2[c] = df2[c].astype("category").cat.codes
        return df2

    Xp = _to_numeric_for_plot(Xs)

    # ---- SHAP main effects（raw で高速安定）----
    explainer = shap.TreeExplainer(model, model_output="raw")
    sv = explainer(Xs)              # shap.Explanation
    S = sv.values                   # (n_samples, n_features)
    F = Xp.values.astype(float)     # (n_samples, n_features)
    cols = X.columns.to_list()
    n, d = S.shape

    # ---- 相関を一括で計算（ベクトル化）----
    # 標準化（分散0は避ける）
    S_std = S.std(axis=0, ddof=0)
    F_std = F.std(axis=0, ddof=0)
    S_std[S_std == 0] = 1.0
    F_std[F_std == 0] = 1.0
    S_z = (S - S.mean(axis=0)) / S_std   # n×d
    F_z = (F - F.mean(axis=0)) / F_std   # n×d

    # 相関行列: d×d（各 i の SHAP と各 j の特徴値との相関）
    # corr = (S_z^T @ F_z) / (n-1)
    C = (S_z.T @ F_z) / max(n - 1, 1)
    C = np.nan_to_num(C, nan=0.0, posinf=0.0, neginf=0.0)
    C = np.abs(C)  # 絶対値がスコア

    # ---- 上三角分だけをCSVに落とす ----
    rows = []
    for i in range(d):
        for j in range(i + 1, d):
            rows.append({"feat_A": cols[i], "feat_B": cols[j], "score": C[i, j]})

    df_scores = (pd.DataFrame(rows)
                   .sort_values("score", ascending=False)
                   .reset_index(drop=True))

    out_csv = os.path.join(OUTPUT_DIR, f"shap_interactions_all_kind{kind}.csv")
    df_scores.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[kind={kind}] 全特徴量ペア {len(df_scores):,} 行の interaction scores を {out_csv} に保存しました。")


#FIXED: 目的変数への寄与度のdependence_plotを追加
def shap_dependence_batch(
    model,
    X,                    # category対応済みの学習用X（= make_X_for_kindの返り値）
    kind,
    sample_size=2000,
    seed=42,
    features="topk",      # "topk" or "all" or list of feature names
    topk=20,              # features="topk"のときの数
    color_by=None,      # "auto" or None or 列名（例："BUILDING_AGE_BIN"）
    out_prefix="shap_depend"
):
    """
    依存プロットを一括保存。
    X軸: 各特徴の値、Y軸: その特徴のSHAP値（目的変数への寄与）
    色: 'auto'→ approximate_interactionsでその特徴と相互作用が強い相手を選択
        列名→ 常にその列で着色、None→着色なし
    """
    # ---- サンプル抽出（重い計算のため） ----
    rng = np.random.RandomState(seed)
    n = min(sample_size, len(X))
    idx = rng.choice(X.index, size=n, replace=False)
    Xs = X.loc[idx].copy()

    # 可視化用に数値化（カテゴリ→codes）
    Xs_plot = to_numeric_for_plot(Xs)

    # ---- SHAP計算（rawで高速安定） ----
    explainer = shap.TreeExplainer(model, model_output="raw")
    sv = explainer(Xs)                         # shap.Explanation
    S = sv.values                              # (n,d)

    # ---- 対象特徴の選定 ----
    all_feats = list(Xs.columns)
    if isinstance(features, list):
        target_feats = [f for f in features if f in all_feats]
    elif features == "all":
        target_feats = all_feats
    else:
        # "topk": |SHAP|平均の上位
        mean_abs = np.abs(S).mean(axis=0)
        target_feats = (pd.Series(mean_abs, index=all_feats)
                        .sort_values(ascending=False).head(topk).index.tolist())

    # ---- 色付けの相手列を決定 ----
    # auto の場合は特徴ごとに approximate_interactions で相手候補を選ぶ
    name_to_idx = {c: i for i, c in enumerate(all_feats)}
    auto_partner = {}
    if color_by == None:
        for f in target_feats:
            order_idx = shap.utils.approximate_interactions(name_to_idx[f], S, Xs_plot)
            # order_idx[0] は自分自身。次点を色付け列に（存在すれば）
            if len(order_idx) > 1:
                auto_partner[f] = all_feats[order_idx[1]]
            else:
                auto_partner[f] = None

    # ---- 図の出力 ----
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    for f in target_feats:
        plt.figure()
        if color_by == None:
            cb = auto_partner.get(f, None)
        elif isinstance(color_by, str):
            cb = color_by if color_by in Xs_plot.columns else None
        else:
            cb = None

        shap.dependence_plot(
            ind=f,
            shap_values=S,
            features=Xs_plot,
            feature_names=Xs_plot.columns,
            interaction_index=(cb if cb in Xs_plot.columns else "auto"),  # ない場合はshapに任せる
            show=False
        )
        plt.tight_layout()
        # ファイル名に色列も入れておくと後で見返しやすい
        suffix = f"_by_{cb}" if cb else ""
        plt.savefig(os.path.join(OUTPUT_DIR, f"{out_prefix}_{f}{suffix}_kind{kind}.png"), dpi=180)
        plt.close()

    # どの列で色付けしたかログCSVも残す
    log_rows = []
    for f in target_feats:
        chosen = (auto_partner.get(f) if color_by == "auto" else (color_by if isinstance(color_by, str) else None))
        log_rows.append({"feature": f, "color_by": chosen})
    pd.DataFrame(log_rows).to_csv(
        os.path.join(OUTPUT_DIR, f"{out_prefix}_colorby_kind{kind}.csv"),
        index=False, encoding="utf-8-sig"
    )
    print(f"[kind={kind}] dependence_plot: {len(target_feats)}枚を書き出し（色指定: {color_by}）")


/opt/anaconda3/envs/marisa/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# === SHAP 実行ループ（修正版） ===
import os, glob, joblib
import pandas as pd

os.makedirs(OUTPUT_DIR, exist_ok=True)  # FIXED: 出力先の作成（初回でもコケないように）

def get_model_for_kind(kind: int):
    """kind=1/2/3 の LightGBM モデルを取得。
    1) model_registry[kind]["model"] があればそれ
    2) MODEL_DIR 内の pkl/joblib を検索してロード
    """
    # 1) 既に学習済みモデルがメモリにある場合
    try:
        if "model_registry" in globals():
            m = model_registry[kind]["model"]
            if m is not None:
                return m
    except Exception:
        pass

    # 2) ファイルからロード
    candidates = []
    # よく使う名前
    candidates += [
        os.path.join(MODEL_DIR, f"lgbm_kind{kind}_v1.pkl"),
        os.path.join(MODEL_DIR, f"lgbm_property_kind{kind}.pkl"),
        os.path.join(MODEL_DIR, f"lgbm_kind{kind}.joblib"),
        os.path.join(MODEL_DIR, f"lgbm_property_kind{kind}.joblib"),
    ]
    # ワイルドカードでも探す
    patterns = [
        os.path.join(MODEL_DIR, f"*kind{kind}*.pkl"),
        os.path.join(MODEL_DIR, f"*kind{kind}*.joblib"),
    ]
    for pat in patterns:
        candidates += glob.glob(pat)

    for path in candidates:
        if os.path.exists(path):
            return joblib.load(path)

    raise FileNotFoundError(
        f"モデルファイルが見つかりませんでした（kind={kind}）。MODEL_DIR={MODEL_DIR} を確認してください。"
    )



In [16]:
import pandas as pd, os

def dump_category_code_map(X, col, out_dir, kind=None):
    """
    X[col] を category にして、code ↔ label の対応表をCSVで保存する。
    - 欠損(NaN)は code = -1 として行を追加
    """
    s = X[col]
    if s.dtype != "category":
        s = s.astype("category")
    cats = list(s.cat.categories)
    df_map = pd.DataFrame({
        "code": list(range(len(cats))),
        "label": cats
    })
    # NaN（欠損）があれば -1 行を付ける
    if s.isna().any():
        df_map = pd.concat([pd.DataFrame([{"code": -1, "label": "<NA>"}]), df_map], ignore_index=True)

    os.makedirs(out_dir, exist_ok=True)
    fname = f"category_codes_{col}.csv" if kind is None else f"category_codes_{col}_kind{kind}.csv"
    path = os.path.join(out_dir, fname)
    df_map.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"saved: {path}")
    return df_map


In [17]:


# ------------------------------------------------------------
# バイオリン型のサマリ図（violinが無ければdotに自動フォールバック）
# 変更点: 上位 max_display の特徴名リストを返すようにした
# ------------------------------------------------------------
def plot_summary_beeswarm_violin(model, X, kind, sample_size=3000, seed=0, max_display=25):
    import numpy as np, shap, matplotlib.pyplot as plt, os
    X = ensure_no_object_category(X)
    n = min(sample_size, len(X))
    idx = np.random.RandomState(seed).choice(X.index, size=n, replace=False)
    Xs = X.loc[idx]
    # 可視化用は数値化（category→codes）
    Xs_plot = Xs.copy()
    for c in Xs_plot.columns:
        if str(Xs_plot[c].dtype).startswith("category"):
            Xs_plot[c] = Xs_plot[c].cat.codes
        elif Xs_plot[c].dtype == "object":
            Xs_plot[c] = Xs_plot[c].astype("category").cat.codes

    explainer = make_shap_explainer_raw(model)
    sv = explainer(Xs)

    # === 追加: 上位特徴名の算出 ===
    # shap.summary_plot は順番を返さないため、ここで mean(|SHAP|) の上位を自前で決定
    mean_abs = np.abs(sv.values).mean(axis=0)
    # 上位 max_display 個の特徴名（図の並びと合うよう降順）
    top_feats = (pd.Series(mean_abs, index=Xs.columns)
                 .sort_values(ascending=False)
                 .head(max_display)
                 .index
                 .tolist())

    plt.figure()
    try:
        shap.summary_plot(sv, Xs_plot, plot_type="violin", max_display=max_display, show=False)
    except Exception:
        shap.summary_plot(sv, Xs_plot, plot_type="dot",    max_display=max_display, show=False)
    plt.tight_layout()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    plt.savefig(os.path.join(OUTPUT_DIR, f"shap_summary_violin_kind{kind}.png"), dpi=180)
    plt.close()

    # === 追加: 使った上位特徴をCSVにも保存（後から再現できるように） ===
    pd.Series(top_feats, name="feature").to_csv(
        os.path.join(OUTPUT_DIR, f"top_features_from_violin_kind{kind}.csv"),
        index=False, encoding="utf-8-sig"
    )

    return top_feats  # ← 追加: 上位リストを返す


# ------------------------------------------------------------
# 単一特徴の依存プロット（色付け列は任意）
# ------------------------------------------------------------
def plot_dependence_one(model, X, kind, feature, color_by=None,
                        sample_size=3000, seed=0, jp_labels=False):
    import numpy as np, shap, matplotlib.pyplot as plt, os
    X = ensure_no_object_category(X)
    n = min(sample_size, len(X))
    idx = np.random.RandomState(seed).choice(X.index, size=n, replace=False)
    Xs = X.loc[idx]
    # 可視化用は数値化
    Xs_plot = Xs.copy()
    for c in Xs_plot.columns:
        if str(Xs_plot[c].dtype).startswith("category"):
            Xs_plot[c] = Xs_plot[c].cat.codes
        elif Xs_plot[c].dtype == "object":
            Xs_plot[c] = Xs_plot[c].astype("category").cat.codes

    explainer = make_shap_explainer_raw(model)
    sv = explainer(Xs)

    plt.figure()
    shap.dependence_plot(
        ind=feature,
        shap_values=sv.values,
        features=Xs_plot,
        feature_names=Xs_plot.columns,
        interaction_index=(color_by if (color_by and color_by in Xs_plot.columns) else None),
        show=False
    )
    if jp_labels:
        plt.xlabel(feature)
        plt.ylabel("SHAP値（成約への寄与度）")
        plt.title(f"Dependence Plot: {feature}")
    plt.tight_layout()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    plt.savefig(os.path.join(OUTPUT_DIR, f"plot_dependence_one_{feature}_kind{kind}.png"), dpi=180)
    plt.close()

def _sanitize(s: str) -> str:
    """ファイル名に使えない文字をアンダースコアへ"""
    return re.sub(r'[\\/:*?"<>|]+', '_', s)




In [18]:


TARGET = TARGET  # 既存の目的変数名（例: "成約フラグ"）。上で定義済みなので流用

from sklearn.inspection import permutation_importance
from sklearn.metrics import get_scorer

def _sample_xy_for_speed(X, y, n=5000, seed=42):
    """計算時間短縮のために最大n件にサンプリング。行数が少なければそのまま返す。"""
    if len(X) <= n:
        return X, y
    rng = np.random.RandomState(seed)
    idx = rng.choice(len(X), size=n, replace=False)
    return X.iloc[idx], y.iloc[idx]

def run_permutation_importance(model, X, y, *, n_repeats=10, seed=42):
    """ROC-AUC を指標に Permutation Importance を計算して返す。"""
    r = permutation_importance(
        model, X, y,
        scoring=get_scorer("roc_auc"),
        n_repeats=n_repeats,
        random_state=seed,
        n_jobs=1,
    )
    df = (pd.DataFrame({
            "feature": X.columns,
            "auc_drop_mean": r.importances_mean,   # 大：その列を壊すとAUCが下がる＝重要
            "auc_drop_std":  r.importances_std,
        })
        .sort_values("auc_drop_mean", ascending=False)
        .reset_index(drop=True)
    )
    return df

def save_perm_bar(imp_df, kind, out_dir, fname_png, topn=25):
    """上位N本の横棒グラフ（AUC drop）を保存。base AUCは描かない。"""
    plot_df = imp_df.head(topn).iloc[::-1]
    plt.figure(figsize=(7, 8))
    plt.barh(plot_df["feature"], plot_df["auc_drop_mean"])
    plt.xlabel("AUC drop (higher = more important)")
    plt.title(f"[kind={kind}] Permutation Importance")
    plt.tight_layout()
    os.makedirs(out_dir, exist_ok=True)
    plt.savefig(os.path.join(out_dir, fname_png), dpi=160)
    plt.close()



def _load_meta_from_disk(kind):
    """
    学習時に保存済みのメタ情報から
    features / cat_cols / test,train index / model を復元。
    （model_registry が無いケースのフォールバック）
    """
    # モデル
    mdl_path_candidates = [
        os.path.join(MODEL_DIR, f"lgbm_kind{kind}.pkl"),
        os.path.join(MODEL_DIR, f"lgbm_property_kind{kind}.pkl"),
        os.path.join(MODEL_DIR, f"lgbm_kind{kind}.joblib"),
        os.path.join(MODEL_DIR, f"lgbm_property_kind{kind}.joblib"),
    ]
    model = None
    for p in mdl_path_candidates:
        if os.path.exists(p):
            model = joblib.load(p); break
    if model is None:
        raise FileNotFoundError(f"model file not found for kind={kind}")

    # メタ（features, cat_cols 等）
    meta = json.load(open(os.path.join(OUTPUT_DIR, f"meta_kind{kind}.json"), "r"))
    feature_cols = meta["features"]
    cat_cols = meta["cat_cols"]

    # split index
    idx_tr = pd.read_csv(os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv"))["train_index"].tolist()
    idx_te = pd.read_csv(os.path.join(OUTPUT_DIR, f"split_test_index_kind{kind}.csv"))["test_index"].tolist()
    return model, feature_cols, cat_cols, idx_tr, idx_te


### 実行 ###

In [19]:
#相互作用の確認

for kind in [1, 2, 3]:
    print(f"[SHAP] property_kind={kind} ...")
    df_kind       = kind_to_df[kind].copy()
    feature_cols  = kind_to_feats[kind]
    category_cols = kind_to_cats[kind]

    model = get_model_for_kind(kind)

    # --- X を作る（学習 & 可視化向けにカテゴリ化） ---
    X_all = make_X_for_kind(df_kind, feature_cols, category_cols)

    # FIXED: SHAP は基本「学習データ」で実施（テストは評価専用）
    #        既に保存済みの学習インデックスがあればそれを使用。無ければ全体でフォールバック。
    split_train_path = os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv")
    if os.path.exists(split_train_path):
        idx_tr = pd.read_csv(split_train_path)["train_index"].tolist()
        # 元DFに存在する行だけに安全に制限
        idx_tr = [i for i in idx_tr if i in X_all.index]
        if len(idx_tr) == 0:
            print(f"  [WARN] split_train_index_kind{kind}.csv は空か不整合。全体にフォールバックします。")
            X_for_shap = X_all
        else:
            X_for_shap = X_all.loc[idx_tr]
    else:
        print(f"  [INFO] split_train_index_kind{kind}.csv が無いので全体で実施します。")
        X_for_shap = X_all

    # --- 既存のサマリ（beeswarm/bar, TopK依存） ---
    shap_all_plots(model, X_for_shap, kind, sample_size=2000, seed=SEED, topk_depend=5, do_interaction=False)
    #                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # FIXED: ここでは相互作用の重い計算は切る（別関数で安定近似をCSV出力するため）

    # FIXED: Top20 を相互作用相手を自動色付けで一括保存
    shap_dependence_batch(
        model, X_for_shap, kind,
        sample_size=2000, seed=SEED,
        features="topk", topk=20,
        color_by=None, out_prefix="shap_depend"
    )

    # --- 全ペア相互作用スコア（安定近似でCSV） ---
    # FIXED: shap_interaction_valuesは使わず、相関ベース近似で全ペアのスコアを保存
    save_interaction_scores(model, X_for_shap, kind, sample_size=2000, seed=SEED)

print("完了。OUTPUT_DIR の CSV/PNG を確認してください。")


[SHAP] property_kind=1 ...


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:62: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, show=False)
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:68: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, plot_type="bar", show=False)
/opt/anaconda3/envs/marisa/lib/python3.13/site-packages/shap/plots/_scatter.py:641: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. 

[kind=1] dependence_plot: 20枚を書き出し（色指定: None）
[kind=1] 全特徴量ペア 1,711 行の interaction scores を /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/shap_interactions_all_kind1.csv に保存しました。
[SHAP] property_kind=2 ...


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:62: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, show=False)
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:68: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, plot_type="bar", show=False)


[kind=2] dependence_plot: 20枚を書き出し（色指定: None）
[kind=2] 全特徴量ペア 1,225 行の interaction scores を /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/shap_interactions_all_kind2.csv に保存しました。
[SHAP] property_kind=3 ...


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [20]:
# ==== Permutation Importance 出力（学習済み前提・後処理／学習データベース） ====
for kind in [1, 2, 3]:
    print(f"[Permutation] property_kind={kind} ...")

    df_kind = kind_to_df[kind].copy()

    # model_registry があれば優先、無ければ保存物から復元
    if "model_registry" in globals() and kind in model_registry:
        model        = model_registry[kind]["model"]
        feature_cols = model_registry[kind]["features"]
        cat_cols     = model_registry[kind]["cat_cols"]
        idx_tr = pd.read_csv(os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv"))["train_index"].tolist()
    else:
        # モデルとメタをディスクから復元
        model = joblib.load(os.path.join(MODEL_DIR, f"lgbm_kind{kind}.pkl"))
        meta  = json.load(open(os.path.join(OUTPUT_DIR, f"meta_kind{kind}.json"), "r"))
        feature_cols = meta["features"]
        cat_cols     = meta["cat_cols"]
        idx_tr = pd.read_csv(os.path.join(OUTPUT_DIR, f"split_train_index_kind{kind}.csv"))["train_index"].tolist()

    # --- 学習データ（train）を母集団にする ---
    X_full = df_kind.reindex(columns=feature_cols).loc[idx_tr].copy()
    # 学習側なのでカテゴリ整合は軽微でOK（unusedを落とすだけ）
    for c in cat_cols:
        if c in X_full.columns:
            X_full[c] = X_full[c].astype("category")
            X_full[c] = X_full[c].cat.remove_unused_categories()

    # 念のため object→category 統一
    X_full = ensure_no_object_category(X_full)
    y_full = df_kind.loc[idx_tr, TARGET].astype(int)

    # --- 5,000件サンプルで Permutation（高速） ---
    X_perm, y_perm = _sample_xy_for_speed(X_full, y_full, n=5000, seed=SEED)
    imp_df = run_permutation_importance(model, X_perm, y_perm, n_repeats=10, seed=SEED)

    # --- 保存（CSVはフル、PNGは上位25のみ） ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    csv_path = os.path.join(OUTPUT_DIR, f"perm_importance_auc_full_kind{kind}.csv")
    png_path = os.path.join(OUTPUT_DIR, f"perm_importance_auc_top25_kind{kind}.png")

    # ▼ 小数第6位に丸め、文字列化して固定表記
    imp_df_fmt = imp_df.copy()
    imp_df_fmt["auc_drop_mean"] = imp_df_fmt["auc_drop_mean"].apply(lambda x: f"{x:.6f}")
    imp_df_fmt["auc_drop_std"]  = imp_df_fmt["auc_drop_std"].apply(lambda x: f"{x:.6f}")

    imp_df_fmt.to_csv(csv_path, index=False, encoding="utf-8-sig")

    save_perm_bar(imp_df, kind, OUTPUT_DIR, os.path.basename(png_path), topn=25)

    print(f"  -> saved: {os.path.basename(csv_path)}, {os.path.basename(png_path)}")




[Permutation] property_kind=1 ...
  -> saved: perm_importance_auc_full_kind1.csv, perm_importance_auc_top25_kind1.png
[Permutation] property_kind=2 ...
  -> saved: perm_importance_auc_full_kind2.csv, perm_importance_auc_top25_kind2.png
[Permutation] property_kind=3 ...
  -> saved: perm_importance_auc_full_kind3.csv, perm_importance_auc_top25_kind3.png


In [21]:
# ===== ここから SHAP 実行ループ =====
for kind in [1, 2, 3]:
    print(f"[SHAP] property_kind={kind} ...")
    df_kind = kind_to_df[kind].copy()
    feature_cols = kind_to_feats[kind]
    category_cols = kind_to_cats[kind]

    model = get_model_for_kind(kind)
    X = make_X_for_kind(df_kind, feature_cols, category_cols)

    # 既存の一括出力（beeswarm / bar / TopK 依存 / 全ペア相関）
    shap_all_plots(model, X, kind, sample_size=2000, seed=SEED, topk_depend=5, do_interaction=True)
    # save_interaction_scores(model, X, kind, sample_size=2000, seed=SEED)

    # =========================
    # FIXED: 追加の2種類の図を保存
    # =========================

    # (A) バイオリン型サマリ（図2）
    # 修正点: ここで上位25特徴のリストを受け取る
    top25_feats = plot_summary_beeswarm_violin(
        model, X, kind, sample_size=3000, seed=SEED, max_display=25
    )

    # 例）PREFECTURE のマッピングを保存
    X_safe = ensure_no_object_category(X)   # ← あなたの関数でobject→categoryに統一
    dump_category_code_map(X_safe, "PREFECTURE", OUTPUT_DIR, kind=kind)

    # (B) 単一特徴の依存プロット（図1）
    # 修正点: ループに順位を付与し、保存直後にリネームして "kind-順位." を先頭に付ける
    for rank, feat in enumerate(top25_feats, start=1):
        if feat in X.columns:
            plot_dependence_one(
                model, X, kind,
                feature=feat,
                color_by=None,            # 色分けなし
                sample_size=3000,
                seed=SEED,
                jp_labels=False
            )
        
            # 例）PREFECTURE のマッピングを保存
            X_safe = ensure_no_object_category(X)   # ← あなたの関数でobject→categoryに統一
            dump_category_code_map(X_safe, "PREFECTURE", OUTPUT_DIR, kind=kind)

            # ▼ ここから追加: リネームで "kind-順位." を先頭に付与
            base = f"plot_dependence_one_{_sanitize(feat)}_kind{kind}.png"
            src  = os.path.join(OUTPUT_DIR, base)
            dst  = os.path.join(OUTPUT_DIR, f"{kind}-{rank}.{base}")
            if os.path.exists(src):
                os.replace(src, dst)   # 既存ファイルがあれば置き換え

    # (C) SHAP 重要度バー（mean(|SHAP|)でソートした自作版）
    # --------------------------------------------------------
    # サンプリングして計算を軽く（必要に応じて件数調整）
    _n = min(3000, len(X))
    _idx = np.random.RandomState(SEED).choice(X.index, size=_n, replace=False)
    Xs_cate = ensure_no_object_category(X.loc[_idx].copy())

    # 可視化用（カテゴリ→codes の数値化は "図の色分け用" だけ。SHAP計算はカテゴリのまま）
    Xs_plot = Xs_cate.copy()
    for c in Xs_plot.columns:
        dt = Xs_plot[c].dtype
        if str(dt).startswith("category"):
            Xs_plot[c] = Xs_plot[c].cat.codes
        elif dt == "object":
            Xs_plot[c] = Xs_plot[c].astype("category").cat.codes

    # SHAP計算（raw + tree_path_dependent）
    explainer = make_shap_explainer_raw(model)
    sv = explainer(Xs_cate)  # shap.Explanation

    # mean(|SHAP|) を算出して降順に
    mean_abs = np.abs(sv.values).mean(axis=0)
    imp_series = pd.Series(mean_abs, index=Xs_cate.columns).sort_values(ascending=False)
    imp_df_shap = (
        imp_series
        .reset_index()
        .rename(columns={"index": "feature", 0: "mean_abs_shap"})
    )

    # 保存（CSVは全特徴、PNGは上位25）
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    csv_path = os.path.join(OUTPUT_DIR, f"shap_importance_meanabs_full_kind{kind}.csv")
    png_path = os.path.join(OUTPUT_DIR, f"shap_importance_meanabs_top25_kind{kind}.png")

    # CSV（数値は固定小数で書き出し）
    imp_df_csv = imp_df_shap.copy()
    imp_df_csv["mean_abs_shap"] = imp_df_csv["mean_abs_shap"].apply(lambda x: f"{x:.6f}")
    imp_df_csv.to_csv(csv_path, index=False, encoding="utf-8-sig")

    # 図（上位25を横棒で、上が最大になるように反転）
    topn = 25
    plot_df = imp_df_shap.head(topn).iloc[::-1]  # 反転して最大を上に
    plt.figure(figsize=(7, 8))
    plt.barh(plot_df["feature"], plot_df["mean_abs_shap"])
    plt.xlabel("mean(|SHAP value|) (average impact on model output magnitude)")
    plt.title(f"[kind={kind}] SHAP Feature Importance (mean abs)")
    plt.tight_layout()
    plt.savefig(png_path, dpi=180)
    plt.close()

    print(f"  -> saved: {os.path.basename(csv_path)}, {os.path.basename(png_path)}")


    # ===== ここまで SHAP 実行ループ =====

[SHAP] property_kind=1 ...


/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:62: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, show=False)
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/10461453.py:68: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(sv, Xs_plot, plot_type="bar", show=False)
/var/folders/g0/2b_vns0j58s8vjrwhmlk88l40000gn/T/ipykernel_13178/3112753872.py:34: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour an

saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv
saved: /Users/okada1015/Desktop/マリサ/marisa/output_

/opt/anaconda3/envs/marisa/lib/python3.13/site-packages/shap/plots/_scatter.py:641: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=figsize)


saved: /Users/okada1015/Desktop/マリサ/marisa/output_model_v1/category_codes_PREFECTURE_kind1.csv


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>